In [5]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.applications import VGG16

SEED = 555

np.random.seed = SEED
tf.random.set_seed(SEED)


In [10]:
train_dir = '/kaggle/input/100-bird-species/train'
# val_dir = '/kaggle/input/100-bird-species/test'
test_dir = '/kaggle/input/100-bird-species/test'


In [12]:
batch_size = 32
img_size = 224


In [13]:
train_datagen = ImageDataGenerator(
    rescale=1/255.,
    samplewise_std_normalization=True,
    rotation_range=45,
    width_shift_range=0.3,
    height_shift_range=0.3,
    brightness_range=(0.1, 0.4),
    shear_range=20.0,
    zoom_range=30.0,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2,
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input
)

val_datagen = ImageDataGenerator(rescale=1/255.,
                                 preprocessing_function=tf.keras.applications.vgg16.preprocess_input
                                 )

test_datagen = ImageDataGenerator(rescale=1/255.,
                                  preprocessing_function=tf.keras.applications.vgg16.preprocess_input
                                  )


/opt/conda/lib/python3.7/site-packages/keras/preprocessing/image.py:1470: UserWarning: This ImageDataGenerator specifies `samplewise_std_normalization`, which overrides setting of `samplewise_center`.
  "This ImageDataGenerator specifies "


In [ ]:
train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                    target_size=(
                                                        img_size, img_size),
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    shuffle=True,
                                                    subset='training',
                                                    seed=SEED
                                                    )

validation_generator = val_datagen.flow_from_directory(directory=train_dir,
                                                       target_size=(
                                                           img_size, img_size),
                                                       batch_size=batch_size,
                                                       shuffle=False,
                                                       class_mode='categorical',
                                                       subset='validation',
                                                       seed=SEED
                                                       )

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(
                                                      img_size, img_size),
                                                  batch_size=batch_size,
                                                  shuffle=False,
                                                  class_mode='categorical')
